In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install vit_keras

In [3]:
pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 4.3 MB/s 


In [4]:
!unzip /content/drive/MyDrive/Graduation_Project/CheXpert-v1.0-small.zip > /dev/null

In [1]:
import cv2
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from myowngen_v2 import DataGenerator
from vit_keras import vit, utils

In [2]:
sample_path  = 'sample_4.csv'
valid_path   = 'val_sample_2.csv' 
data_path    = "/content/"
weights_path = '/content/drive/MyDrive/Graduation_Project/model_weight.hdf5'

In [3]:
train = pd.read_csv(sample_path)
#train.drop(columns=['Binary'], inplace=True)
train.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00057/study2/...,Female,48,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,CheXpert-v1.0-small/train/patient00060/study1/...,Female,44,Frontal,PA,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,CheXpert-v1.0-small/train/patient00060/study1/...,Female,44,Lateral,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,CheXpert-v1.0-small/train/patient00066/study1/...,Male,61,Frontal,PA,1.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
4,CheXpert-v1.0-small/train/patient00066/study1/...,Male,61,Lateral,NaN,1.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [4]:
train.loc[:, train.columns[5:]] = train.loc[:, train.columns[5:]].fillna(0)

to_take = list(set(train.columns[5:])-set(['Edema', 'Atelectasis']))
train.loc[:, to_take] = train.loc[:, to_take].replace({-1:0})

train.loc[:, ['Edema', 'Atelectasis']] = train.loc[:, ['Edema', 'Atelectasis']].replace({-1:1})

In [5]:
valid = pd.read_csv(valid_path)
valid.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,Sex_y
0,CheXpert-v1.0-small/train/patient04947/study3/...,Male,56,Frontal,PA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
1,CheXpert-v1.0-small/train/patient38193/study3/...,Male,67,Frontal,AP,0,0,0,1,0,1,0,0,0,0,0,0,0,1,NaN
2,CheXpert-v1.0-small/train/patient47458/study7/...,Male,57,Frontal,AP,0,0,0,1,0,0,0,0,0,0,0,0,1,1,NaN
3,CheXpert-v1.0-small/train/patient38830/study3/...,Male,56,Frontal,AP,0,0,0,0,0,0,0,0,1,0,0,0,0,1,NaN
4,CheXpert-v1.0-small/train/patient26417/study1/...,Female,50,Lateral,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN


In [6]:
valid.drop('Sex_y', axis=1, inplace=True)

In [10]:
# valid.loc[:, valid.columns[5:]] = valid.loc[:, valid.columns[5:]].fillna(0)

# to_take = list(set(valid.columns[5:])-set(['Edema', 'Atelectasis']))
# valid.loc[:, to_take] = valid.loc[:, to_take].replace({-1:0})

# valid.loc[:, ['Edema', 'Atelectasis']] = valid.loc[:, ['Edema', 'Atelectasis']].replace({-1:1})

In [7]:
train_generator = DataGenerator(data_path, train, 14, batch_size=32, shape=(224,224, 3), shuffle=True)

In [8]:
valid_generator = DataGenerator(data_path, valid, 14, batch_size=32, shape=(224,224, 3), shuffle=True)

In [9]:
checkpoint = ModelCheckpoint(weights_path, monitor='val_auc', verbose=1, save_best_only=False, mode='auto', save_freq='epoch')
early = EarlyStopping(monitor="val_auc", mode='auto', patience=5, restore_best_weights=False)
callbacks_list = [checkpoint, early]

In [10]:
base_model = vit.vit_b16(image_size=224,
                        pretrained=True,
                        pretrained_top = False,
                        include_top=False)

/usr/local/lib/python3.7/dist-packages/vit_keras/utils.py:83: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  UserWarning,


In [15]:
#base_model.layers

In [11]:
for layer in base_model.layers[:-5]:
    layer.trainable = False

for layer in base_model.layers[-5:]:
    layer.trainable = True

#building the model
model=Sequential()
model.add(base_model)
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(14,activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit-b16 (Functional)        (None, 768)               85798656  
                                                                 
 dense (Dense)               (None, 512)               393728    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 14)                1806      
                                                                 
Total params: 86,259,854
Trainable params: 21,726,350
No

In [12]:
model.load_weights(weights_path)

In [13]:
INIT_LR, EPOCHS, BS = 0.0001, 4, 32
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=INIT_LR, decay= INIT_LR/EPOCHS),
              metrics=['binary_accuracy', tf.keras.metrics.AUC(multi_label = True)])

In [19]:
history = model.fit(train_generator, validation_data=valid_generator, epochs=1, batch_size=BS, verbose=1, callbacks=callbacks_list)

1883/1883 [==============================] - ETA: 0s - loss: 0.3335 - binary_accuracy: 0.8587 - auc: 0.8089
Epoch 00001: saving model to /content/drive/MyDrive/Graduation_Project/model_weight.hdf5
1883/1883 [==============================] - 2938s 2s/step - loss: 0.3335 - binary_accuracy: 0.8587 - auc: 0.8089 - val_loss: 0.3199 - val_binary_accuracy: 0.8632 - val_auc: 0.7475


# Test

In [15]:
from sklearn.metrics import roc_auc_score

def custom_auc(y_true, y_pred):
    labels = ['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']

    results = pd.DataFrame(index=labels)


    scores = []
    for i in [2, 5, 6, 8, 10]:
        score = roc_auc_score(y_true[:, i], y_pred[:, i])
        scores.append(score)
        
    results['AUC'] = scores

    return results

In [16]:
val_path   = '/content/CheXpert-v1.0-small/valid.csv'
val = pd.read_csv(val_path)

In [17]:
#train_generator = DataGenerator(data_path, train, 14, batch_size=1, shape=(224,224, 3), shuffle=False)
val_generator = DataGenerator(data_path, val, 14, batch_size=1, shape=(224,224, 3), shuffle=False)

In [18]:
# actual
y_val_true   = val.iloc[:, 5:].values

# predicted
y_val_pred   = model.predict(val_generator)

results = custom_auc(y_val_true, y_val_pred)
results.AUC.mean()

0.8411446416967863

In [ ]:
# actual
y_train_true   = train.iloc[:, 5:].values

# predicted
y_train_pred   = model.predict(train_generator)

results = custom_auc(y_train_true, y_train_pred)
results.AUC.mean()

0.7247363198395431